In [1]:
!date

Wed Aug 10 12:25:37 EDT 2022


In [2]:
pwd

'/mmfs1/data/pijarj/BC-ORG-Data/Code'

In [3]:
cd ../Data

/mmfs1/data/pijarj/BC-ORG-Data/Data


In [4]:
%%time
import pandas as pd
import os
import shutil
import numpy as np
from tqdm import tqdm
import ants
import nibabel as nib

CPU times: user 1.65 s, sys: 800 ms, total: 2.45 s
Wall time: 23.5 s


In [5]:
#make txt file into csv
read_file = pd.read_csv ('/mmfs1/data/pijarj/NDAR_BoldAnat10/image03.txt',delimiter='\t',low_memory=False)
read_file.to_csv ('/mmfs1/data/pijarj/BC-ORG-Data/Data/image03.csv', index=0)

In [6]:
#slice general csv into dataset

# Drops non nii or nii.gz types
df = pd.read_csv('image03.csv',low_memory=False)
df = df.iloc[1::] # First row is just another index, let's drop it
df['image_file'] = df['image_file'].astype(str) # convert entries to strings (so nan become 'nan')

df['is_nii_gz'] = [str(file).endswith('.nii.gz') for file in df['image_file'].values]
df['is_nii'] = [str(file).endswith('.nii') for file in df['image_file'].values]

# only keep the niftis 
df = df.iloc[df['is_nii_gz'].values + df['is_nii'].values] #

#study_name = 'Biomarkers of Developmental Trajectories and Treatment in ASD'
study_name = 'Cognitive and Neural Flexibility in Autism'
#study_name = 'Multimodal Imaging of Early Neural Signature in Autism Spectrum Disorder'
#study_name = 'Functional Anatomic Studies of Self-Affect:  A Multimodal Approach'
#study_name = 'A Longitudinal MRI Study of Infants at Risk for Autism (ACE 2)'
#study_name = 'Intrinsic Brain Architecture of Young Children with Autism While Awake and Asleep'
#study_name = 'Using complex video stimuli to elucidate atypical brain functioning in ASD'
#study_name = 'Minimally Verbal ASD: From Basic Mechanisms to Innovative Interventions'
study_df = df[df['collection_title']==study_name]

if len(study_df)==0:
    print('no usable scans')
else:
    col_id = study_df['collection_id'].values[0]
    print(col_id)
    study_df.to_csv(f'../Data/DS{col_id}.csv')
    print(f'../Data/DS{col_id}.csv')
    print(len(study_df))
    print(len(study_df['subjectkey'].unique()))

2342
../Data/DS2342.csv
192
91


In [7]:
cd ../../

/mmfs1/data/pijarj


In [8]:
mkdir 'ds-{col_id}'

mkdir: cannot create directory ‘ds-2342’: File exists


In [9]:
cd ~/BC-ORG-Data/Data

/mmfs1/data/pijarj/BC-ORG-Data/Data


In [10]:
# df = pd.read_csv(f'../Data/DS{col_id}.csv')
# df['is_nii_gz'] = [file.endswith('.nii.gz') for file in df['image_file'].values]
# df = df[df['is_nii_gz']]

In [11]:
def safe_mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        pass

In [12]:
def write_json(data,filepath):
    import json
    with open(filepath, 'w') as outfile:
        json.dump(data, outfile)

In [13]:
#check for unique subjects with both func and anat
#study_df = pd.read_csv(f'../Data/DS{col_id}.csv')
study_subjects = np.unique(study_df['subjectkey'].values)
nsubjects = len(study_subjects)
print(f'{nsubjects} unique subjects')
def check_has_anat_and_epi(sub):
    sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
    fmri_idx = sub_df['scan_type'].values=='fMRI'
    #anat_idx = sub_df['scan_type'].values in ['MR structural (T1)','MR structural (MPRAGE)']
    anat_idx = sub_df['scan_type'].isin(['MR structural (T1)','MR structural (MPRAGE)']).values
    return fmri_idx.sum()>0 and anat_idx.sum()>0

91 unique subjects


In [14]:
ndar_root = '/mmfs1/data/pijarj/NDAR_BoldAnat10/' 
bids_root = '/mmfs1/data/pijarj/'

In [15]:
has_anat_and_epi = np.array([check_has_anat_and_epi(s) for s in study_subjects])

In [16]:
#find subjects with anat and func
use_subjects = study_subjects[has_anat_and_epi]
nsubjects = len(use_subjects)
print(f'{nsubjects} subjects with anat + fmri')

85 subjects with anat + fmri


In [17]:
# MAKE THE BIDS DIRECTORY
safe_mkdir(os.path.join(bids_root,f'DS{col_id}.csv'))
for s in tqdm(range(1,nsubjects+1)):
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}'))
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}','func'))
    safe_mkdir(os.path.join(bids_root,f'ds-{col_id}',f'sub-{s:03d}','anat'))

  0%|                                                                                                                                 | 0/85 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 1022.88it/s]

In [18]:
#making local paths column
#df = pd.read_csv(f'DS{col_id}.csv')
n = len(study_df)
#root = './image03/'
root = os.path.expanduser('~/NDAR_BoldAnat10/image03/')
local_paths = list()
for i in tqdm(range(n)):
    s3_path = study_df['image_file'].values[i]
    splits = s3_path.split('/')[4::]
    relative = '/'.join(splits)
    local_path = os.path.join(root,relative)
    assert os.path.exists(os.path.join(root,relative))
    local_paths.append(local_path)
study_df['local_paths'] = local_paths

  0%|                                                                                                                                | 0/192 [00:00<?, ?it/s]

 50%|███████████████████████████████████████████████████████████                                                           | 96/192 [00:00<00:00, 953.57it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [00:00<00:00, 1119.18it/s]

In [19]:
for s in tqdm(range(0,nsubjects)):
    sub = use_subjects[s]
    sub_df = study_df.iloc[study_df['subjectkey'].values==sub]
    fmri_idx = sub_df['scan_type'].values=='fMRI'
#    anat_idx = sub_df['scan_type'].values=='MR structural (T1)'
    anat_idx = sub_df['scan_type'].isin(['MR structural (T1)','MR structural (MPRAGE)']).values

    epi_fn = sub_df.iloc[fmri_idx]['local_paths'].values[0]
    anat_fn = sub_df.iloc[anat_idx]['local_paths'].values[0]

#     #epi_path = os.path.join(ndar_root,epi_fn[1::])
#     #anat_path = os.path.join(ndar_root,anat_fn[1::])
    
    epi_path = os.path.join(ndar_root,epi_fn)
    anat_path = os.path.join(ndar_root,anat_fn)

    
#     # AA: Is this a problem? we're reading in .nii or .nii.gz but saving as .nii.gz always
    epi_dest = os.path.join(bids_root,f'ds-{col_id}',f'sub-{s+1:03d}','func',f'sub-{s+1:03d}_task-rest_bold.nii.gz')
    anat_dest = os.path.join(bids_root,f'ds-{col_id}',f'sub-{s+1:03d}','anat',f'sub-{s+1:03d}_T1w.nii.gz')

    t1 = ants.image_read(anat_path) 
    bold = ants.image_read(epi_path)

    t1.to_filename(anat_dest)
    bold.to_filename(epi_dest)

#     bold_json = {"RepetitionTime" : bold.spacing[-1],
#                 "TaskName" : 'rest'}
#     write_json(bold_json,epi_dest.replace('.nii.gz','.json')) 
#     im = nib.load(epi_dest)
#     header = im.header.copy()
#     header.set_xyzt_units(xyz='mm', t='sec')
#     nib.nifti1.Nifti1Image(im.get_fdata(), None, header=header).to_filename(epi_dest)
#     assert nib.load(epi_dest).header.get_xyzt_units()==('mm', 'sec'),'timing missing from header'

  0%|                                                                                                                                 | 0/85 [00:00<?, ?it/s]

  1%|█▍                                                                                                                       | 1/85 [00:18<25:35, 18.27s/it]

  2%|██▊                                                                                                                      | 2/85 [00:41<29:21, 21.22s/it]

  4%|████▎                                                                                                                    | 3/85 [00:57<25:52, 18.93s/it]

  5%|█████▋                                                                                                                   | 4/85 [01:18<26:39, 19.75s/it]

...

 44%|████████████████████████████████████████████████████▏                                                                   | 37/85 [12:56<17:19, 21.65s/it]

 45%|█████████████████████████████████████████████████████▋                                                                  | 38/85 [13:16<16:43, 21.35s/it]

 46%|███████████████████████████████████████████████████████                                                                 | 39/85 [13:27<13:49, 18.03s/it]

 47%|████████████████████████████████████████████████████████▍                                                               | 40/85 [13:48<14:22, 19.16s/it]

 48%|█████████████████████████████████████████████████████████▉                                                              | 41/85 [14:08<14:04, 19.20s/it]

In [ ]:
import json
data = {
    "Name" : study_df["collection_title"].values[0] ,
    "RepetitionTime": 2.0,
    "SliceTiming" : 2.0 ,
    "TaskName" : "taskrest" ,
    "BIDSVersion" : "20.2.0"}
json_string = json.dumps(data)
print(json_string)
with open(os.path.join(bids_root,f'ds-{col_id}','dataset_description.json'), 'w') as outfile:
    json.dump(json_string, outfile)

In [ ]:
pwd